In [1]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

Apply Filters (Notch + Bandpass)

In [ ]:
fs = 256  # Sampling frequency in Hz

# Load EEG data
# -------------------------
EEG_raw = np.load('...')

# EEG shape: (n_channels, n_samples)


# Notch filter to remove 50 Hz power line noise
# -------------------------
b_notch, a_notch = signal.iirnotch(50, Q=30, fs=fs)    # Q factor determines the bandwidth of the notch filter
EEG_notched = signal.filtfilt(b_notch, a_notch, EEG_raw, axis=1)


# Band-pass filter between 8-30 Hz
# -------------------------
b_bp, a_bp = signal.cheby2(
    order=4,                    # Higher order = sharper cutoff
    rs=40,                      # Stopband attenuation in dB -> Suppress everything outside the passband by at least rs dB
    Wn=[8/(fs/2), 30/(fs/2)],   # Normalized cutoff frequencies (Nyquist frequency = fs/2)
    btype='bandpass'
)

# Apply zero-phase filtering
EEG_bp = signal.filtfilt(b_bp, a_bp, EEG_notched, axis=1) # Apply filter along time axis (1) -> shape: (channels, timepoints)


Plot EEG Signals

In [ ]:
# Plot results
# -------------------------
channel = 0        # Channel to plot
t = np.arange(EEG_raw.shape[1]) / fs  # Time vector

plt.figure(figsize=(14, 6))

plt.plot(t, EEG_raw[channel], label="Raw EEG", alpha=0.6)   # Original signal
plt.plot(t, EEG_notched[channel], label="Notch filtered (50 Hz)", alpha=0.8)    # Notch filtered signal
plt.plot(t, EEG_bp[channel], label="Notch + Band-pass (8–30 Hz)", linewidth=2)  # Band-pass filtered signal

plt.xlabel("Time (s)")
plt.ylabel("Amplitude (µV)")
plt.title(f"EEG Channel {channel + 1}")
plt.legend()
plt.tight_layout()
plt.show()